# HTTP request

九種請求的方法：

* GET     - retrieving information from a website.
* POST    - sending information (user login data) to a website.
* PUT     - sending information (user login data) to a website.
* DELETE
* HEAD
* CONNECT
* TRACE
* OPTIONS
* PATCH

```html
GET /index.html HTTP/1.1
Host: example.com
```

# HTTP response

## 1. Status line
    
    HTTP/1.1 200 OK

## 2. Response headers

    Cache-Control: max-age=604800
    Content-Type: text/html; charset=UTF-8
    Date: Fri, 16 Aug 2019 07:09:55 GMT
    Etag: "1541025663+gzip"
    Expires: Fri, 23 Aug 2019 07:09:55 GMT
    Last-Modified: Fri, 09 Aug 2013 23:54:35 GMT
    Server: ECS (oxr/8321)
    Vary: Accept-Encoding
    X-Cache: HIT

## 3. Response body

長得像這樣：

```html
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    ...(略)   
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domain is established to be used for illustrative examples in documents. You may use this
    domain in examples without prior coordination or asking for permission.</p>
    <p><a href="http://www.iana.org/domains/example">More information...</a></p>
</div>
</body>
</html>
```

# HTTP status code

用來提供 client 端關於 HTTP request 的狀態，有以下幾種範圍：

## 100~199 

    ⇒ 關於「如何遞送請求」的資訊，通常由 Client 端自行處理，無須理會。
    ⇒ 101 :: switching protocols

## 200~299 

    ⇒ 知會 client 端，關於請求是否處理成功
    ⇒ 200 :: 有 response body 即將到來，一切完美！
    ⇒ 206 :: 常見於下載部分的(chunks)大檔案時，知會 client 端，伺服端正回傳原始檔案的部分內容。

## 300~399 

    ⇒ 代表已經收到請求，但是需要而外的步驟，以取得真正內容。
    ⇒ 301,302,307,308 :: 所請求的資源可以在另一個位置被取得。

    HTTP/1.1 301 Moved Permanently
    Location: /blogs/index.html

## 400~499

    ⇒ 要特別注意這範圍的狀態碼，它們指出 request 的某個地方出錯了！
    ⇒ 404 :: Not Found
    ⇒ 401 :: UNauthorized
    ⇒ 403 :: Forbidden
    ⇒ 429 :: Too Many Requests
    
## 500~599

    ⇒ 指出伺服端有異狀。建議稍後重新嘗試請求！
    ⇒ 502 :: Bad Gateway
    ⇒ 503 :: Service Temporarily Unavailable
    
# HTTP request/response in Go

* Using the `net/http` package in Go's standard library
* The underlying HTTP request:
    
```
GET /index.html HTTP/1.1
Host: example.com
```
* `http.Response` structure:

```go
type Response struct {
    Status     string // e.g. "200 OK"
    
    StatusCode int    // e.g. 200
    
    Proto      string // e.g. "HTTP/1.0"
    ProtoMajor int    // e.g. 1
    ProtoMinor int    // e.g. 0

    Header Header
    Body io.ReadCloser
    
    ContentLength int64Transfer
    Encoding []string
    Close bool
    Uncompressed bool
    Trailer Header
    Request *Request
    TLS *tls.ConnectionState 
}
```

`StatusCode`, `Header`, `Body` 資訊在爬蟲時都非常有用！

In [1]:
import(
    "log"
    "net/http"
    "os"
)

In [2]:
{
//     var r *http.Response
    
    r, err := http.Get("http://www.example.com/index.html")
    if err != nil {
        log.Fatalln(err)
        return
    }
    
    if r.StatusCode == 200 {
        var content []byte
        var bodyLen int = 1270
        
        content = make([]byte, bodyLen)
        r.Body.Read(content)
        
        out, err := os.OpenFile("index.html", 
                                os.O_CREATE | os.O_WRONLY, 0644)
        if err != nil {
            log.Fatalln(err)
            return
        }
        
        out.Write(content)
        out.Close()
    } else {
        log.Fatal("Failed to retrieve the webpage. Received status code:", r.Status)
    }
}